In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("../data/raw/estimaciones-y-proyecciones-2002-2035-comunas.xlsx").iloc[:-2]

# Cálculo de Personas por Estrato

Aquí se quiere calcular la cantidad de personas que hay en los siguientes estratos:

1. Hombres y Mujeres
2. Hombres
3. Mujeres
4. Población sobre 45 años
5. Población sobre 60 años
6. Población entre 60 a 70 años
7. Población recién nacidos

Para cada estrato se calculará la población que hay para el SSMO, Región Metropolitana y el País.

## 1. Hombres y Mujeres

In [11]:
COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
]

poblacion_ssmo = df["Nombre Comuna"].isin(COMUNAS_SSMO)
suma_poblacion_ssmo = df[poblacion_ssmo].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()

poblacion_rm = (df["M"] == "Metropolitana de Santiago")
suma_poblacion_rm = df[poblacion_rm].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()

suma_poblacion_chile = df.loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()

## 2. Hombres

In [12]:
poblacion_ssmo_hombre = (poblacion_ssmo) & (df["Sexo\n1=Hombre\n2=Mujer"] == 1.0)
suma_poblacion_ssmo_hombre = (
    df[poblacion_ssmo_hombre].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_rm_hombre = (poblacion_rm) & (df["Sexo\n1=Hombre\n2=Mujer"] == 1.0)
suma_poblacion_rm_hombre = (
    df[poblacion_rm_hombre].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_pais_hombre = df["Sexo\n1=Hombre\n2=Mujer"] == 1.0
suma_poblacion_chile_hombre = (
    df[poblacion_pais_hombre].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

## 3. Mujeres

In [15]:
poblacion_ssmo_mujer = (poblacion_ssmo) & (df["Sexo\n1=Hombre\n2=Mujer"] == 2.0)
suma_poblacion_ssmo_mujer = (
    df[poblacion_ssmo_mujer].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_rm_mujer = (poblacion_rm) & (df["Sexo\n1=Hombre\n2=Mujer"] == 2.0)
suma_poblacion_rm_mujer = (
    df[poblacion_rm_mujer].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_pais_mujer = df["Sexo\n1=Hombre\n2=Mujer"] == 2.0
suma_poblacion_chile_mujer = (
    df[poblacion_pais_mujer].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)